In [25]:
import audio_processing as ap
import audio_utils as au
import prototype as pr
import math
import numpy.fft as fft
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

instrs = ["guitar", "clarinet", "flute", "saxophone", "violin"]

In [30]:
prototypes = []
exemplars = []
ind = 0
for instr in instrs:
    exemplars.append([])
    infile = open("reps/" + instr + "_harmonics.txt", 'r')
    for line in infile:
        ex = []
        for i in line.split(','):
            ex.append( float(i.strip()) )
        exemplars[ind].append(ex)
    infile.close()
    p = pr.create_prototype(exemplars[ind])
    prototypes.append(p)
    ind += 1
print(np.array(prototypes) )
    

[[ 1.          0.09854176  0.12267083  0.17575874  0.18015025  0.21846029
   0.18708441  0.24378326]
 [ 1.          0.0018178   0.00184637  0.00180747  0.00187048  0.00187077
   0.00174611  0.00193864]
 [ 1.          0.05749046  0.04627587  0.03685894  0.03542063  0.03864822
   0.03609048  0.03916336]
 [ 1.          0.00502223  0.00537474  0.00445526  0.00404386  0.00334559
   0.00284447  0.00362899]
 [ 1.          0.05308918  0.04507265  0.03393457  0.03121332  0.03255566
   0.02535387  0.02753111]]


In [36]:
def euclid_d(a, b):
    d = 0
    for i in range(len(a)):
        d += (a[i] - b[i])**2
    return math.sqrt(d)

def predict(example, prototypes):
    ds = []
    for p in prototypes:
        ds.append(euclid_d(example, p))
    return np.argmin(ds)

In [43]:
guesses = []

for i in range(5):
    guesses.append([])
    for ex in exemplars[i]:
        guess = predict(ex, prototypes)
        guesses[i].append(guess)

In [50]:
for i in range(len(guesses)):
    correct = 0
    tot = len(guesses[i])
    for j in guesses[i]:
        if j == i:
            correct += 1
    print(correct / tot)
    
a = [0,0,0,0,0]
for i in guesses[4]:
    a[i] += 1
print(a)

0.7183098591549296
0.8571428571428571
0.06914212548015365
0.10628019323671498
0.12319644839067703
[53, 137, 116, 484, 111]
